In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Apr_PM")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [2]:
travel_demand

8x8 Array{Float64,2}:
   0.0    364.031  325.364  387.088   543.744   531.093  533.457  187.046
 511.458    0.0    326.3    459.388   581.073   663.681  610.212  213.57 
 302.447  331.95     0.0    636.435  1224.59   1315.28   831.621  334.268
 190.454  163.849  276.317    0.0     278.569   362.274  272.226  127.833
 374.333  399.123  573.729  472.954     0.0     777.604  600.51   260.003
 324.129  461.955  608.26   648.385   754.479     0.0    735.143  254.818
 431.09   512.002  714.337  517.884   849.503  1621.38     0.0    188.789
 239.612  234.682  514.833  383.71    402.015   566.538  282.776    0.0  

In [3]:
############
#Read in the demand file
file = open("East_Massachusetts_trips_Apr_PM.txt")
# demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
#                 demands[(s,t)] = demand 
                travel_demand[s,t] = demand 
            end
        end
    end
end                
close(file)

In [4]:
travel_demand

8x8 Array{Float64,2}:
   0.0    364.031  325.364  387.088   543.744   531.093  533.457  187.046
 511.458    0.0    326.3    459.388   581.073   663.681  610.212  213.57 
 302.447  331.95     0.0    636.435  1224.59   1315.28   831.621  334.268
 190.454  163.849  276.317    0.0     278.569   362.274  272.226  127.833
 374.333  399.123  573.729  472.954     0.0     777.604  600.51   260.003
 324.129  461.955  608.26   648.385   754.479     0.0    735.143  254.818
 431.09   512.002  714.337  517.884   849.503  1621.38     0.0    188.789
 239.612  234.682  514.833  383.71    402.015   566.538  282.776    0.0  

In [13]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e4
    l = 1
    #average_excess_cost = 1
    tol = 1e-6

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [15]:
bpr_coeffs = [1, 0, 0, 0, .15]

xl = MSA(bpr_coeffs)

24-element Array{Float64,1}:
 2871.82 
  701.911
    0.0  
 1671.61 
 4515.54 
 1939.71 
  846.475
  354.302
 5696.02 
 4270.06 
 2510.05 
 1394.34 
 1189.39 
 2275.66 
 1139.88 
 1402.86 
  127.833
  383.71 
 3842.91 
 4415.96 
  989.962
 2187.92 
 1438.49 
 2240.46 

In [16]:
sum([free_flow_time[a]*xl[a] + .03*free_flow_time[a]*((xl[a])^5)/((capacity[a])^4) for a = 1:24])

10788.719821327722

In [17]:
# getting the coefficients of the costs
coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["(8,0.5,10000.0,1)"]

xl = MSA(fcoeffs)

24-element Array{Float64,1}:
 2871.82 
  701.911
    0.0  
 1671.61 
 4515.54 
 1939.71 
  846.475
  354.302
 5696.02 
 4270.06 
 2510.05 
 1394.34 
 1189.39 
 2275.66 
 1139.88 
 1402.86 
  127.833
  383.71 
 3842.91 
 4415.96 
  989.962
 2187.92 
 1438.49 
 2240.46 

In [18]:
sum([free_flow_time[a]*xl[a] + .03*free_flow_time[a]*((xl[a])^5)/((capacity[a])^4) for a = 1:24])

10788.719821327722